In [74]:
import pandas as pd
import numpy as np

from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [75]:
fname = 'pred_data.json'
d1 = pd.read_json(fname)
d1.head()

,lat,lng,likelihood
0,-4.538094,39.385986,0.005912
1,-4.398444,39.504089,0.008661
2,-4.228639,39.580994,0.005912
3,-4.034138,39.666138,0.005912
4,-3.891658,39.776001,0.005912


In [76]:
d1['lat']=d1['lat'].astype('float')
d1['lng']=d1['lng'].astype('float')
d1['likelihood']=d1['likelihood'].astype('float')*10
d1=d1.reset_index()
d1=d1.drop('index',axis=1)
d1.head()

,lat,lng,likelihood
0,-4.538094,39.385986,0.059116
1,-4.398444,39.504089,0.086615
2,-4.228639,39.580994,0.059116
3,-4.034138,39.666138,0.059116
4,-3.891658,39.776001,0.059116


In [77]:
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df=wgs84_to_web_mercator(d1,'lng','lat')


In [78]:
scale=2000
x=df['x']
y=df['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 350))
x_max=int(x.mean() + (scale * 350))
y_min=int(y.mean() - (scale * 350))
y_max=int(y.mean() + (scale * 350))

#Defining the map tiles to use in this case OSM

tile_provider=get_provider(OSM)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='Power Outage Likelihoods for Today',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator'
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False

output_notebook()

Loading BokehJS ...

In [79]:
#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map. 
def bubble_map(plot,df,radius_col,lon,lat,scale,color='orange',leg_label='Bubble Map'):
  radius=[]
  for i in df[radius_col]:
    radius.append(i*scale)
  
  df['radius']=radius
    
  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  lat_label='@'+lat
  lon_label='@'+lon

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [80]:
#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
bubble_map(plot=plot,
           df=d1,
           radius_col='likelihood', 
           leg_label='Power Outage Likelihoods for today',
           lon='lng',
           lat='lat',
           scale=scale)

show(plot)

In [81]:
save(plot, 'likelihoods.html')

C:\Anaconda\envs\myenv1\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Anaconda\envs\myenv1\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\Andy\\Desktop\\SCI\\4th Year\\4th YEAR PROJECT\\likelihoods.html'